In [1]:
!pip install pyspark==3.4.0

In [ ]:
!pip install findspark pyarrow fastparquet

In [ ]:
# Import and initialise findspark
import findspark
findspark.init()

In [ ]:
# Start Spark session
from pyspark.sql import SparkSession
import time

spark = SparkSession.builder.appName("SparkSQL").getOrCreate()

In [ ]:
# 1. Read in the AWS S3 bucket into a DataFrame.
from pyspark import SparkFiles
url = "https://2u-data-curriculum-team.s3.amazonaws.com/dataviz-classroom/v1.2/22-big-data/home_sales_revised.csv"
spark.sparkContext.addFile(url)
df = spark.read.csv(SparkFiles.get("home_sales_revised.csv"), sep=",", header=True)
df.show()

+--------------------+----------+----------+------+--------+---------+-----------+--------+------+----------+----+
|                  id|      date|date_built| price|bedrooms|bathrooms|sqft_living|sqft_lot|floors|waterfront|view|
+--------------------+----------+----------+------+--------+---------+-----------+--------+------+----------+----+
|f8a53099-ba1c-47d...|2022-04-08|      2016|936923|       4|        3|       3167|   11733|     2|         1|  76|
|7530a2d8-1ae3-451...|2021-06-13|      2013|379628|       2|        2|       2235|   14384|     1|         0|  23|
|43de979c-0bf0-4c9...|2019-04-12|      2014|417866|       2|        2|       2127|   10575|     2|         0|   0|
|b672c137-b88c-48b...|2019-10-16|      2016|239895|       2|        2|       1631|   11149|     2|         0|   0|
|e0726d4d-d595-407...|2022-01-08|      2017|424418|       3|        2|       2249|   13878|     2|         0|   4|
|5aa00529-0533-46b...|2019-01-30|      2017|218712|       2|        3|       196

In [ ]:
# 2. Create a temporary view of the DataFrame.
df.createOrReplaceTempView('home_sales')

In [ ]:
# 3. What is the average price for a four bedroom house sold in each year rounded to two decimal places?
spark.sql("""
SELECT
  YEAR(date) as Year,
  ROUND(AVG(price),2) as avgPrice
FROM home_sales
WHERE bedrooms =4
GROUP BY YEAR(date)
ORDER BY YEAR(date) asc
""").show()

+----+---------+
|Year| avgPrice|
+----+---------+
|2019| 300263.7|
|2020|298353.78|
|2021|301819.44|
|2022|296363.88|
+----+---------+



In [ ]:
# 4. What is the average price of a home for each year the home was built that have 3 bedrooms and 3 bathrooms rounded to two decimal places?
spark.sql("""
SELECT
  date_built,
  ROUND(AVG(price),2) as avgPrice
FROM home_sales
WHERE bedrooms = 3 and bathrooms = 3
GROUP BY date_built
ORDER BY date_built asc
""").show()

+----------+---------+
|date_built| avgPrice|
+----------+---------+
|      2010|292859.62|
|      2011|291117.47|
|      2012|293683.19|
|      2013|295962.27|
|      2014|290852.27|
|      2015| 288770.3|
|      2016|290555.07|
|      2017|292676.79|
+----------+---------+



In [ ]:
# 5. What is the average price of a home for each year built that have 3 bedrooms, 3 bathrooms, with two floors,
# and are greater than or equal to 2,000 square feet rounded to two decimal places?
spark.sql("""
SELECT
  date_built as dateBuilt,
  ROUND(AVG(price),2) as avgPrice
FROM home_sales
WHERE bedrooms = 3 and bathrooms = 3 and floors = 2 and sqft_living >= 2000
GROUP BY date_built
ORDER BY date_built asc
""").show()

+---------+---------+
|dateBuilt| avgPrice|
+---------+---------+
|     2010|285010.22|
|     2011|276553.81|
|     2012|307539.97|
|     2013|303676.79|
|     2014|298264.72|
|     2015|297609.97|
|     2016| 293965.1|
|     2017|280317.58|
+---------+---------+



In [ ]:
# 6. What is the "view" rating for the average price of a home, rounded to two decimal places, where the homes are greater than
# or equal to $350,000? Although this is a small dataset, determine the run time for this query.

start_time = time.time()

spark.sql("""
SELECT
  view,
  ROUND(AVG(price),2) as avgPrice
FROM home_sales
GROUP BY view
HAVING AVG(price) >= 350000
ORDER BY view asc
""").show()


print("--- %s seconds ---" % (time.time() - start_time))

+----+---------+
|view| avgPrice|
+----+---------+
| 100|1026669.5|
|  51|788128.21|
|  52|733780.26|
|  53| 755214.8|
|  54|798684.82|
|  55|771153.32|
|  56| 718176.4|
|  57| 734340.5|
|  58|759764.65|
|  59| 791453.0|
|  60|754939.65|
|  61|746877.59|
|  62|759150.14|
|  63|711614.55|
|  64|767036.67|
|  65|736679.93|
|  66| 712475.0|
|  67|737970.96|
|  68|716785.44|
|  69|750537.94|
+----+---------+
only showing top 20 rows

--- 0.5446789264678955 seconds ---


In [ ]:
# 7. Cache the the temporary table home_sales.
spark.sql('cache table home_sales')

DataFrame[]

In [ ]:
# 8. Check if the table is cached.
spark.catalog.isCached('home_sales')

True

In [ ]:
# 9. Using the cached data, run the query that filters out the view ratings with average price
#  greater than or equal to $350,000. Determine the runtime and compare it to uncached runtime.

start_time = time.time()

spark.sql("""
SELECT
  view,
  ROUND(AVG(price),2) as avgPrice
FROM home_sales
GROUP BY view
HAVING AVG(price) < 350000
ORDER BY view asc
""").show()

print("--- %s seconds ---" % (time.time() - start_time))

+----+---------+
|view| avgPrice|
+----+---------+
|   0|285069.21|
|   1|288248.11|
|  10|289693.49|
|  11|280356.07|
|  12|286575.22|
|  13|285849.95|
|  14|282519.32|
|  15|284907.04|
|  16|291990.83|
|  17|286601.22|
|  18|287532.36|
|  19|280347.62|
|   2|287710.06|
|  20|284820.43|
|  21|283451.36|
|  22|284908.42|
|  23|286977.37|
|  24|281878.86|
|  25|288583.21|
|  26|281224.95|
+----+---------+
only showing top 20 rows

--- 0.3635215759277344 seconds ---


In [ ]:
# 10. Partition by the "date_built" field on the formatted parquet home sales data
df.write.partitionBy('date_built').mode('overwrite').parquet('home_sales_partitioned')

In [ ]:
# 11. Read the parquet formatted data.
df_p=spark.read.parquet('home_sales_partitioned')

In [ ]:
# 12. Create a temporary table for the parquet data.
df_p.createOrReplaceTempView('home_sales_p')

In [ ]:
# 13. Run the query that filters out the view ratings with average price of greater than or equal to $350,000
# with the parquet DataFrame. Round your average to two decimal places.
# Determine the runtime and compare it to the cached version.

start_time = time.time()

spark.sql("""
SELECT
  view,
  ROUND(AVG(price),2) as avgPrice
FROM home_sales_p
GROUP BY view
HAVING AVG(price) < 350000
ORDER BY view asc
""").show()

print("--- %s seconds ---" % (time.time() - start_time))

+----+---------+
|view| avgPrice|
+----+---------+
|   0|285069.21|
|   1|288248.11|
|  10|289693.49|
|  11|280356.07|
|  12|286575.22|
|  13|285849.95|
|  14|282519.32|
|  15|284907.04|
|  16|291990.83|
|  17|286601.22|
|  18|287532.36|
|  19|280347.62|
|   2|287710.06|
|  20|284820.43|
|  21|283451.36|
|  22|284908.42|
|  23|286977.37|
|  24|281878.86|
|  25|288583.21|
|  26|281224.95|
+----+---------+
only showing top 20 rows

--- 0.8624935150146484 seconds ---


In [ ]:
# 14. Uncache the home_sales temporary table.
spark.sql('uncache table home_sales')

DataFrame[]

In [ ]:
# 15. Check if the home_sales is no longer cached
spark.catalog.isCached('home_sales')

False